In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

data_path = 'adult/adult.data'
column_names = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation', 
              'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
             'wage_class']
data = pd.read_csv(data_path, names=column_names, sep = ",", skipinitialspace=True,  header=None)
data["wage_class"] = data["wage_class"].map({"<=50K": 0, ">50K": 1})
data.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,wage_class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [3]:
data["workclass"] = data["workclass"].replace({"Never-worked": "?", "Without-pay": "?"})
data.sex = data.sex.replace({"Male": 0, "Female": 1})
data.capital_gain = (data.capital_gain > 0).astype(int)
data.capital_loss = (data.capital_loss > 0).astype(int)
data.native_country = (data.native_country == "United-States").astype(int)

C:\Users\mlempicki\AppData\Local\Temp\ipykernel_16756\559932236.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.sex = data.sex.replace({"Male": 0, "Female": 1})


In [4]:
data = data.drop(["education"], axis=1)
data.rename({"native_country": "us_native"}, axis=1, inplace=True)
data = data.drop(["fnlwgt"], axis=1)

In [5]:
data.head()

,age,workclass,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,us_native,wage_class
0,39,State-gov,13,Never-married,Adm-clerical,Not-in-family,White,0,1,0,40,1,0
1,50,Self-emp-not-inc,13,Married-civ-spouse,Exec-managerial,Husband,White,0,0,0,13,1,0
2,38,Private,9,Divorced,Handlers-cleaners,Not-in-family,White,0,0,0,40,1,0
3,53,Private,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,0,0,0,40,1,0
4,28,Private,13,Married-civ-spouse,Prof-specialty,Wife,Black,1,0,0,40,0,0


In [6]:
X = data.drop(["wage_class"], axis=1)
y = data.wage_class

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Definiujemy grupy zmiennych i pipeliny

In [8]:
num_columns = ["age", "education_num", "hours_per_week"]
cat_columns = ["workclass", "marital_status", "occupation", "relationship", "race"]
bin_columns = ["sex", "capital_gain", "capital_loss", "us_native"]

In [9]:
num_pipe = Pipeline(
    ("scaler", StandardScaler())
)
cat_pipe = Pipeline(
    ("encoder", OneHotEncoder(drop="first", handle_unknown="ignore"))
)

In [10]:
from sklearn.compose import ColumnTransformer

In [23]:
preprocessor = ColumnTransformer([
    ("num", num_pipe, num_columns),
    ("cat", cat_pipe, cat_columns),
    ("bin", "passtrough", bin_columns)
])

preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=('scaler', StandardScaler())),
                                 ['age', 'education_num', 'hours_per_week']),
                                ('cat',
                                 Pipeline(steps=('encoder',
                                                 OneHotEncoder(drop='first',
                                                               handle_unknown='ignore'))),
                                 ['workclass', 'marital_status', 'occupation',
                                  'relationship', 'race']),
                                ('bin', 'passtrough',
                                 ['sex', 'capital_gain', 'capital_loss',
                                  'us_native'])])

In [24]:
pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("model", LogisticRegression())
])
pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=('scaler',
                                                                  StandardScaler())),
                                                  ['age', 'education_num',
                                                   'hours_per_week']),
                                                 ('cat',
                                                  Pipeline(steps=('encoder',
                                                                  OneHotEncoder(drop='first',
                                                                                handle_unknown='ignore'))),
                                                  ['workclass',
                                                   'marital_status',
                                                   'occupation', 'relationship',
                                                   'race']),
                                                 ('bin', 'passtrough',
                                                  ['sex', 'capital_gain',
                                                   'capital_loss',
                                                   'us_native'])])),
                ('model', LogisticRegression())])

In [27]:
preprocessor__num__

NameError: name 'preprocessor__num__' is not defined

In [14]:
preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=('scaler', StandardScaler())),
                                 ['age', 'education_num', 'hours_per_week']),
                                ('cat',
                                 Pipeline(steps=('encoder',
                                                 OneHotEncoder(drop='first',
                                                               handle_unknown='ignore'))),
                                 ['workclass', 'marital_status', 'occupation',
                                  'relationship', 'race']),
                                ('bin', 'passtrough',
                                 ['sex', 'capital_gain', 'capital_loss',
                                  'us_native'])])

In [15]:
pipeline.named_steps["preprocessor"]

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=('scaler', StandardScaler())),
                                 ['age', 'education_num', 'hours_per_week']),
                                ('cat',
                                 Pipeline(steps=('encoder',
                                                 OneHotEncoder(drop='first',
                                                               handle_unknown='ignore'))),
                                 ['workclass', 'marital_status', 'occupation',
                                  'relationship', 'race']),
                                ('bin', 'passtrough',
                                 ['sex', 'capital_gain', 'capital_loss',
                                  'us_native'])])

In [16]:
pipeline.named_steps["preprocessor"].transformers_[1][1].steps[0][1].categories_

AttributeError: 'ColumnTransformer' object has no attribute 'transformers_'

In [20]:
pipeline.fit(X_train, y_train)
pipeline.predict(X_test)

TypeError: All estimators should implement fit and transform, or can be 'drop' or 'passthrough' specifiers. 'Pipeline(steps=('scaler', StandardScaler()))' (type <class 'sklearn.pipeline.Pipeline'>) doesn't.

In [ ]:
accuracy_score(y_test, pipeline.predict(X_test))

In [ ]:
y.mean()

### Zoptymalizuj siłę regularyzcji w reg. log. w tym pipelinie. Na skali logarytmicznej od 10^-5 do 10^5. Wykorzystaj: np.logspace.
#### Pamiętaj od zapewnienieu losowania kolejności w podziale na foldy. Przetestować na danych testowych.

In [18]:
np.logspace(-5, 5, 11)

array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05])

In [28]:
param_grid = {
    "model__C": np.logspace(-5, 5, 11)
}

cv = KFold(10, shuffle=True)
optimizer = GridSearchCV(pipeline, param_grid, cv=cv, scoring="accuracy", n_jobs=-1)
optimizer.fit(X_train, y_train)

print(optimizer.best_params_)

accuracy_score(y_test, optimizer.best_estimator_.predict(X_test))

ValueError: 
All the 110 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
110 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\mlempicki\AI_ML\venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\mlempicki\AI_ML\venv\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\mlempicki\AI_ML\venv\Lib\site-packages\sklearn\pipeline.py", line 469, in fit
    Xt = self._fit(X, y, routed_params)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\mlempicki\AI_ML\venv\Lib\site-packages\sklearn\pipeline.py", line 406, in _fit
   